In [9]:
#В этом файле изучается создание классификационной модели
# Подключим все необходимые библиотеки
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [10]:
# Библиотеки из обучающего видео

import nltk #анализ текста
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import PrecisionRecallDisplay

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
nltk.download('stopwords') #Загрузка стоп-слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
#Загрузим датасет с размеченными данными
#В датасете информация о негативных комментариях


data_path = '/content/drive/MyDrive/Colab Notebooks/MLdatasets/labeled.csv'

df = pd.read_csv(data_path)
df.head(10)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [13]:
#Возьму код из своей прошлой практики для разделения данных на тестовые и обучающие
train_df, test_df = train_test_split(df, test_size = 0.2)

In [14]:
sentence_example = df.iloc[1]["comment"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


#Функция обработки

In [15]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [16]:
tokenize_sentence(sentence_example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

#Обучение модели

In [17]:
vectorizer = TfidfVectorizer(tokenizer=tokenize_sentence)

In [18]:
#Делим все слова как признаки предложений
features = vectorizer.fit_transform(train_df["comment"])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
#Обучаем модель определять по словам токсичность комментария
model = LogisticRegression(random_state=0)
model.fit(features, train_df["toxic"])

LogisticRegression(random_state=0)

In [20]:
# Проверим первый коммент
model.predict(features[0])

array([0.])

In [21]:
#Прочитаем его
train_df["comment"].iloc[0]

'7 лет есть гарантия у них. Но чёт не стал брать. Взял на пробу с трехлетней, доверия к ним до этого упало, дал шанс вернуть его\n'

# Создание пайплайна модели(делаем модель рабочей у пользователя)

In [22]:
#Мы объединяем векторайзер и модель, тем самым мы создаем модель, которая работает с цельными предложениями
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = tokenize_sentence)),
    ("model", LogisticRegression(random_state=0))
]
)
#Все из-за того, что мы до этого дробили предложения

In [23]:
model_pipeline.fit(train_df["comment"], train_df["toxic"])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function tokenize_sentence at 0x78f95ecebf40>)),
                ('model', LogisticRegression(random_state=0))])

#Измерим метрики модели

In [24]:
precision_score(y_true=test_df["toxic"], y_pred=model_pipeline.predict(test_df["comment"]))

0.8691176470588236

In [25]:
recall_score(y_true=test_df["toxic"], y_pred=model_pipeline.predict(test_df["comment"]))

0.635483870967742

In [26]:
import joblib

joblib.dump(model_pipeline, "model.pkl")

['model.pkl']